In [6]:
debug = False

from picamera import PiCamera
from picamera.array import PiRGBArray, PiYUVArray
from scipy.signal import filtfilt, butter, find_peaks
from pwm import PWM
import sys
from time import sleep
import threading
import ipywidgets as widgets
from ivPID import PID

In [7]:
minSpeed =  1000000 # 0 speed
zeroRot  =  1000000
maxRot   =   500000
rot = zeroRot


pwm0 = PWM(0)  # motor
# pwm0.export()
pwm0.period = 20000000
pwm0.duty_cycle = minSpeed

pwm1 = PWM(1)  # servo
# pwm1.export() 
pwm1.period = 20000000
pwm1.duty_cycle = zeroRot 

pwm0.enable = True
pwm1.enable = True

if not debug:
    print("waiting for moter")
    sleep(3)
    print("motor ready")

waiting for moter
motor ready


In [8]:
roi_perc = 0.8
peaks_threshold = 0.9
threshold_deviation = 0.15
kp = 2500
ki = 0
kd = 0
speed = 1150000
terminate = False


def set_r_s(r):
    global roi_perc
    roi_perc = r["new"]

def set_p_s(p):
    global peaks_threshold
    peaks_threshold = p["new"]
    
def set_t_s(t):
    global threshold_deviation
    threshold_deviation = t["new"]  

def set_k_s(k):
    global kp
    kp = k["new"]      

def set_kd_s(k):
    global kd
    kd = k["new"]

def set_ki_s(k):
    global ki
    ki = k["new"]

def set_s_s(s):
    global speed
    speed = s["new"] 

def set_o_s(o):
    global terminate
    terminate = o["new"] 


r_s = widgets.FloatSlider(description="ROI line", value=0.8, min=0.0, max=1.0)
p_s = widgets.FloatSlider(description="Peak threshold", value=0.9, min=0.0, max=1.0)
t_s = widgets.FloatSlider(description="Threshold deviation", value=0.15, min=0.0, max=1.0)
k_s = widgets.IntSlider(description="Kp", value=2500, min=0, max=4000, step = 50)
ki_s = widgets.IntSlider(description="Ki", value=0, min=0, max=4000, step = 50)
kd_s = widgets.IntSlider(description="Kd", value=0, min=0, max=4000, step = 50)
s_s = widgets.IntSlider(description="Speed", value=1200000, min=1000000, max=2000000, step=50000)
o_s = widgets.Checkbox(description="Terminate", value=False, disabled=False)

r_s.observe(set_r_s, "value")
p_s.observe(set_p_s, "value")
t_s.observe(set_t_s, "value")
k_s.observe(set_k_s, "value")
ki_s.observe(set_ki_s, "value")
kd_s.observe(set_kd_s, "value")
s_s.observe(set_s_s, "value")
o_s.observe(set_o_s, "value")
display(r_s, p_s, t_s, k_s, ki_s, kd_s, s_s, o_s)
# kp 1150
# ki 450
# kd 0

FloatSlider(value=0.8, description='ROI line', max=1.0)

FloatSlider(value=0.9, description='Peak threshold', max=1.0)

FloatSlider(value=0.15, description='Threshold deviation', max=1.0)

IntSlider(value=2500, description='Kp', max=4000, step=50)

IntSlider(value=0, description='Ki', max=4000, step=50)

IntSlider(value=0, description='Kd', max=4000, step=50)

IntSlider(value=1200000, description='Speed', max=2000000, min=1000000, step=50000)

Checkbox(value=False, description='Terminate')

In [9]:
# kp should be around 2500.
# ki
# get the error. Error is peak position - middle.
# servo angle is 1.5 million + kp*error
def t_func():
    global rot
    global roi_perc
    global peaks_threshold
    global threshold_deviation
    global kp
    global ki
    global kd
    global terminate
    global speed
    
    rawCapture = PiYUVArray(camera)
    stream = camera.capture_continuous(rawCapture, format="yuv", use_video_port=True)
    b, a = butter(3, 0.007)
    pid = PID.PID(kp, ki, kd)
    pid.SetPoint=0.0
    pid.setSampleTime(0.01)
    try:
        for frame in stream:
            pid.setKp(kp)
            pid.setKi(ki)
            pid.setKd(kd)
            I = frame.array[:, :, 0]
            rawCapture.truncate(0)

            mid_horizontal = int(I.shape[0]*roi_perc)
            mid_vertical = int(I.shape[1]/2)

            L = I[mid_horizontal, :]
            Lf = filtfilt(b, a, L)

            # normalize LF
            max_Lf = max(Lf)
            min_Lf = min(Lf)
            Lf = [val/(max_Lf - min_Lf) for val in Lf]

            p = find_peaks(Lf, peaks_threshold)

            closest_peak_at = -1
            distance_peak = 1000000
            for peak in p[0]:
                if abs(mid_vertical - peak) < distance_peak:
                    closest_peak_at = peak
                    distance_peak = abs(mid_vertical - peak)
        
#             deviation = ((mid_vertical - closest_peak_at)*1.0)/mid_vertical

#             if deviation < -1*threshold_deviation:
#                 move = "right"
#             elif -1*threshold_deviation <= deviation <= threshold_deviation:
#                 move = "straight"
#             else:
#                 move = "left"
            error = mid_horizontal - closest_peak_at
            pid.update(error)
            new_angle = zeroRot - pid.output
            new_angle = max(zeroRot - maxRot, new_angle)
            new_angle = min(zeroRot + maxRot, new_angle)
            rot = new_angle
#             if move == 'left':
#                 rot = min(zeroRot + maxRot, rot+0.10*rot)
#             elif move == 'right':
#                 rot = max(zeroRot -maxRot, rot-0.10*rot)
#             else:
#                 rot = zeroRot

#             print(speed, roi_perc, peaks_threshold, threshold_deviation)
            pwm0.duty_cycle = speed
            pwm1.duty_cycle = int(rot)
            if terminate:
                print("YAY! TERMINATING !!!!")
                pwm0.duty_cycle = minSpeed
                pwm1.duty_cycle = zeroRot
                stream.close()
                rawCapture.close()
                camera.close()
                break
    except:
        pwm0.duty_cycle = minSpeed
        pwm1.duty_cycle = zeroRot
        stream.close()
        rawCapture.close()
        camera.close()

In [10]:
camera = PiCamera()

t = threading.Thread(target = t_func)
t.start()